In [1]:
import os
import numpy as np
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from sensorium.data import data
from sensorium.utils import utils, tensorboard

utils.set_random_seed(1234)

GRAY = cm.get_cmap("gray")
BACKGROUND_COLOR = "#fbfbfb"

In [24]:
dataset, metadata = data.load_mice_data(mice_dir="../data", mouse_ids=None)

Loading: 100%|██████████| 7/7 [01:07<00:00,  9.61s/it]


In [ ]:
for i in dataset.keys():
    print(
        f'Mouse {i+1}\t\tnum. neurons: {dataset[i]["response"].shape[1]}\t\tnum. trials: {dataset[i]["image"].shape[0]}'
    )

In [ ]:
# normalize array to range [0, 1]
normalize = lambda a: (a - np.min(a)) / (np.max(a) - np.min(a))
# standardize array to 0 mean and unit variance
standardize = lambda a: (a - np.mean(a)) / np.std(a)

Show response statistics

In [30]:
for mouse_id in dataset.keys():
    print(f"\nMouse {mouse_id}")
    for tier in ["train", "validation", "test"]:
        idx = np.where(metadata[mouse_id]["tiers"] == tier)[0]
        response = dataset[mouse_id]["response"][idx]
        print(
            f"Tier: {tier}\n"
            f"min: {np.min(response):04f}, max: {np.max(response):04f}\n"
            f"mean: {np.mean(response):04f}, std: {np.std(response):04f}"
        )


Mouse 0
Tier: train
min: -0.000000, max: 869.680420
mean: 4.782653, std: 21.239862
Tier: validation
min: -0.000000, max: 840.908569
mean: 4.943315, std: 21.604088
Tier: test
min: 0.000000, max: 0.000000
mean: 0.000000, std: 0.000000

Mouse 1
Tier: train
min: -0.000001, max: 892.844055
mean: 7.185532, std: 27.497597
Tier: validation
min: -0.000001, max: 882.874756
mean: 7.093803, std: 27.266296
Tier: test
min: 0.000000, max: 0.000000
mean: 0.000000, std: 0.000000

Mouse 2
Tier: train
min: -0.000000, max: 506.363403
mean: 1.372593, std: 6.070605
Tier: validation
min: -0.000000, max: 440.794800
mean: 1.284734, std: 5.746231
Tier: test
min: -0.000000, max: 448.827576
mean: 1.283185, std: 5.755345

Mouse 3
Tier: train
min: -0.000000, max: 706.448914
mean: 3.714447, std: 14.127764
Tier: validation
min: -0.000000, max: 593.421509
mean: 3.646763, std: 13.742233
Tier: test
min: -0.000000, max: 723.797363
mean: 3.588892, std: 13.687257

Mouse 4
Tier: train
min: -0.000000, max: 701.732483
mean: 

Plot mouse 3 responses

In [ ]:
responses = normalize(dataset[2]["response"])

figure, ax = plt.subplots(
    nrows=1,
    ncols=1,
    gridspec_kw={"wspace": 0.05, "hspace": 0.01},
    figsize=(5, 5),
    dpi=120,
    facecolor=BACKGROUND_COLOR,
)


N_TRIALS, N_NEURONS = 60, 40
ax.imshow(responses[:N_TRIALS, :N_NEURONS].T, cmap=GRAY)

x_ticks = np.linspace(0, N_TRIALS - 1, 5)
tensorboard.set_xticks(
    axis=ax, ticks_loc=x_ticks, ticks=x_ticks.astype(int), label="Trials"
)
y_ticks = np.linspace(0, N_NEURONS - 1, 5)
tensorboard.set_yticks(
    axis=ax, ticks_loc=y_ticks, ticks=y_ticks.astype(int), label="Neurons"
)

# add colorbar
width, height = 0.015, ax.get_position().y1 * 0.3
cbar_ax = figure.add_axes(
    rect=[
        ax.get_position().x1 + 0.01,
        ((ax.get_position().y1 - ax.get_position().y0) / 2 + ax.get_position().y0)
        - (height / 2),
        width,
        height,
    ]
)
figure.colorbar(cm.ScalarMappable(cmap=GRAY), cax=cbar_ax, shrink=0.1)
tensorboard.set_yticks(
    axis=cbar_ax,
    ticks_loc=np.linspace(0, 1, 3),
    ticks=np.linspace(0, 1, 3),
    label="",
)
ax.set_title("Mouse 3 responses vs images")

tensorboard.save_figure(
    figure, filename="plots/mouse3_responses_v_images.png", dpi=240, close=False
)

plt.show()
plt.close(figure)

Plot image - response pairs from Mouse 3

In [ ]:
N_TRIALS = 3

images, responses, behaviors = (
    dataset[2]["image"],
    dataset[2]["response"],
    dataset[2]["behavior"],
)
print(
    f"image dimension: {images.shape[1:]}\n" f"number of neurons: {responses.shape[1]}"
)

figure, axes = plt.subplots(
    nrows=N_TRIALS,
    ncols=2,
    gridspec_kw={"wspace": 0.015, "hspace": 0.2},
    figsize=(8, 3 * N_TRIALS),
    dpi=120,
    facecolor=BACKGROUND_COLOR,
)

trials = np.random.choice(images.shape[0], size=N_TRIALS, replace=False)
x_axis = np.arange(responses.shape[1])

for i, trial in enumerate(trials):
    # plot response
    axes[i, 0].scatter(x=x_axis, y=responses[trial], s=2, alpha=0.8, color="orangered")
    axes[i, 0].set_facecolor(BACKGROUND_COLOR)
    axes[i, 0].set_ylabel(f"Trial {trial:03d}\nResponse")
    if i == N_TRIALS - 1:
        axes[i, 0].set_xlabel("Neurons")
    tensorboard.remove_top_right_spines(axis=axes[i, 0])
    axes[i, 0].set_ylim(np.min(responses[trial]), np.max(responses[trial]))
    # plot natural image
    axes[i, 1].imshow(normalize(images[trial][0]), cmap=GRAY, aspect="auto")
    axes[i, 1].set_xlabel(
        rf"pupil size: {behaviors[trial][0]:.02f}, $\Delta$ size: {behaviors[trial][1]:.02f}, speed: {behaviors[trial][2]:.02f}"
    )
    axes[i, 1].set_xticks([])
    axes[i, 1].set_yticks([])
    axes[i, 1].tick_params(axis="both", which="both", pad=0, length=0)
    tensorboard.remove_spines(axis=axes[i, 1])

tensorboard.save_figure(
    figure, filename="plots/mouse3_responses_v_images_2.png", dpi=240, close=False
)

plt.show()
plt.close(figure)

Plot Mouse 3 - 7 response distributions

In [ ]:
N_TRIALS = 100

figure, ax = plt.subplots(
    nrows=1,
    ncols=1,
    gridspec_kw={"wspace": 0.05, "hspace": 0.05},
    figsize=(1.5 * len(dataset), 5),
    dpi=120,
    facecolor=BACKGROUND_COLOR,
)

responses = [dataset[i]["response"][:N_TRIALS].flatten() for i in dataset.keys()]
sns.violinplot(
    data=responses,
    scale="area",
    width=0.9,
    inner="quartile",
    linewidth=1,
    ax=ax,
)
ax.set_facecolor(BACKGROUND_COLOR)
ax.set_yscale("symlog")
ax.set_xticklabels(
    [f"Mouse {i+1}" for i in range(len(dataset))], va="top", ha="center", rotation=90
)
ax.set_xlim(left=-0.5, right=len(responses) - 0.5)

tensorboard.remove_top_right_spines(axis=ax)
ax.set_ylabel("Responses (symmetric-log scale)")
ax.set_title("Responses distribution")

tensorboard.save_figure(
    figure, filename="plots/response_distribution.png", dpi=240, close=False
)

plt.show()
plt.close(figure)

Plot anatomical coordinates for mouse 2

In [ ]:
def plot_coordinates(coordinates, mouse_id: int):
    print(
        f"coordinates min: {np.min(coordinates, axis=0)}\n"
        f"coordinates max: {np.max(coordinates, axis=0)}"
    )

    figure = plt.figure(figsize=(6, 6), dpi=120, facecolor=BACKGROUND_COLOR)
    ax = figure.add_subplot(1, 1, 1, projection="3d", facecolor=BACKGROUND_COLOR)
    ax.scatter3D(*coordinates.T, s=10, alpha=0.4, color="black")
    ax.set_xlabel("x coordinate")
    ax.set_ylabel("y coordinate")
    ax.set_zlabel("z coordinate")
    ax.set_title(f"Mouse {mouse_id+1} cell motor coordinates")
    figure.tight_layout()

    tensorboard.save_figure(
        figure,
        filename=f"plots/mouse{mouse_id}_cell_coordinates.png",
        dpi=240,
        close=False,
    )

    plt.show()
    plt.close(figure)

In [ ]:
plot_coordinates(metadata[0]["coordinates"], mouse_id=0)

In [ ]:
plot_coordinates(metadata[1]["coordinates"], mouse_id=1)

In [ ]:
plot_coordinates(metadata[2]["coordinates"], mouse_id=2)

In [ ]:
plot_coordinates(metadata[3]["coordinates"], mouse_id=3)

In [ ]:
plot_coordinates(metadata[4]["coordinates"], mouse_id=4)

In [ ]:
plot_coordinates(metadata[5]["coordinates"], mouse_id=5)

In [ ]:
plot_coordinates(metadata[6]["coordinates"], mouse_id=6)